In [866]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Connect to local llama_server

In [867]:
from ollama import Client
from IPython.display import display_markdown

client = Client(host='http://localhost:11435')

In [868]:
model_name = 'qwen2.5-coder:1.5b'

In [869]:
# Pull a model
result = client.pull(model=model_name)
result

{'status': 'success'}

## Create reflection agent

In [870]:
generation_chat_history = []
reflection_chat_history = []

#### Pre-set the generation agent with a context and request

In [871]:
generation_prompt = {
    "role": "system",
    "content": "You are a experienced Python programmer tasked with generating high quality and very well documented Python code. If the user provides critique, respond with a revised version of your code from previous attempts.",
}

request_prompt = {
    "role": "user",
    "content": "Generate a python code with an implementation of a binary tree inversion.",
}

In [872]:
generation_chat_history.append(generation_prompt)
generation_chat_history.append(request_prompt)

#### Generate a response to the request given the context.

In [873]:
# Generate
generation_response = client.chat(model=model_name, messages=generation_chat_history)

generation_response_content = generation_response['message']['content']

# Assign the reponse the an entity called 'assistant'.
request_reponse_prompt = {
    "role": "assistant",
    "content": generation_response_content,
}

# Add the LLM response to the chat history.
generation_chat_history.append(request_reponse_prompt)

In [874]:
display_markdown(generation_response_content, raw=True)

Sure! Here is an example code for inverting a binary tree:
```python
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def invert_tree(node):
    if not node:
        return
    # Swap left and right children
    node.left, node.right = node.right, node.left
    
    # Invert the left subtree
    invert_tree(node.left)
    
    # Invert the right subtree
    invert_tree(node.right)
```
This code defines a `Node` class that represents a node in the binary tree. The `invert_tree` function takes a `Node` object as input and recursively swaps the left and right children of the node. It also calls itself to invert the left and right subtrees.
To use this code, you can create a new instance of the `Node` class and pass it to the `invert_tree` function:
```python
root = Node(1)
root.left = Node(2)
root.right = Node(3)
root.left.left = Node(4)
root.left.right = Node(5)
root.right.left = Node(6)
root.right.right = Node(7)

print("Before inversion:")
pre_order_traversal(root)  # Output: 1, 2, 4, 5, 3, 6, 7

invert_tree(root)

print("\nAfter inversion:")
pre_order_traversal(root)  # Output: 7, 6, 5, 4, 3, 2, 1
```
In this example, we first create a binary tree with some values. We then call the `invert_tree` function to invert the tree and print the results before and after inversion using the `pre_order_traversal` function.

#### Pre-set the reflection agent with a context and request

In [875]:
reflection_prompt = {
    "role": "system",
    "content": "You are a experienced Python developer reponsible to supervise a software development project. You're tasked for creating and performing tests, identifying errors, and providing detailed feedback and recommendations to the user's code. Critique the code submitted for code quality, completeness, responsability, scope acucary and documentation.",
}

critique_prompt = {
    "role": "user",
    "content": request_response,
}


reflection_chat_history.append(reflection_prompt)
reflection_chat_history.append(critique_prompt)

#### Generate a critique to the generated code.

In [876]:
critique_response = client.chat(model=model_name, messages=reflection_chat_history)
critique_response_content = critique_response['message']['content']

In [877]:
generation_chat_history.append({
    "role": "user",
    "content": critique_response_content
})

In [880]:
generation_chat_history

[{'role': 'system',
  'content': 'You are a experienced Python programmer tasked with generating high quality and very well documented Python code. If the user provides critique, respond with a revised version of your code from previous attempts.'},
 {'role': 'user',
  'content': 'Generate a python code with an implementation of a binary tree inversion.'},
 {'role': 'assistant',
  'content': 'Sure! Here is an example code for inverting a binary tree:\n```python\nclass Node:\n    def __init__(self, value):\n        self.value = value\n        self.left = None\n        self.right = None\n\ndef invert_tree(node):\n    if not node:\n        return\n    # Swap left and right children\n    node.left, node.right = node.right, node.left\n    \n    # Invert the left subtree\n    invert_tree(node.left)\n    \n    # Invert the right subtree\n    invert_tree(node.right)\n```\nThis code defines a `Node` class that represents a node in the binary tree. The `invert_tree` function takes a `Node` objec

In [881]:
best_response = client.chat(model=model_name, messages=generation_chat_history)

In [882]:
final_response = best_response["message"]["content"]

display_markdown(final_response, raw=True)

I completely agree with your suggestions! Here's the final version of the code with the requested improvements:

```python
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def invert_tree(root):
    """
    Inverts a binary tree.

    Args:
    root (TreeNode): The root node of the binary tree to be inverted.

    Returns:
    TreeNode: The modified root node of the inverted binary tree.
    """
    if not root:
        return None 

    temp = root.left
    root.left = root.right
    root.right = temp 

    invert_tree(root.left)
    invert_tree(root.right)

    return root

# Example usage:
# Create a sample binary tree
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

print("Original Tree:")
# Function to print the tree in order for testing purposes (not part of the original code)
def print_tree(node, level=0):
    if not node:
        return
    print_tree(node.right, level+1)
    print(' ' * 4 * level + str(node.val))
    print_tree(node.left, level+1)

print_tree(root)

# Invert the tree
root = invert_tree(root)

print("\nInverted Tree:")
print_tree(root)
```

This improved code is more readable and follows good practices.

In [854]:
reflection_chat_history

[{'role': 'system',
  'content': "You are a experienced Python developer reponsible to supervise a software development project. You're tasked for creating and performing tests, identifying errors, and providing detailed feedback and recommendations to the user's code. Critique the code submitted for code quality, completeness, responsability, scope acucary and documentation."},
 {'role': 'user',
  'content': "Here is a simple Python code implementing a binary tree inversion:\n\n```python\nclass TreeNode:\n    def __init__(self, val=0, left=None, right=None):\n        self.val = val\n        self.left = left\n        self.right = right\n\ndef invert_tree(root):\n    if not root: \n        return None \n\n    temp = root.left\n    root.left = root.right\n    root.right = temp \n\n    invert_tree(root.left)\n    invert_tree(root.right)\n\n    return root\n\n```\nIn this code, we first define a class TreeNode to represent each node in the binary tree. Each node has three attributes: `val`